In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
import os
import re
from pathlib import Path
import ast

In [ ]:
pathlist = Path('../../asc_data_free_viewing').rglob('*.asc')
task_data_df = []
df_final = pd.DataFrame()
# for all participant
for path in pathlist:
    participant= re.search(r"(P\d+)", os.path.basename(str(path.stem))).group(1)
    f = open(str(path), 'r') # 'r' = read
    print(participant)
    lines = f.read()
    f.close()
    # j = 4
    # n = 30
    indices = []
    current_start_index = 0
    not_list = []
    for i in range(1):

        index_init1 = re.search(r'free_viewing_experiment_start',lines[current_start_index:-1]).span()[1]
        index_init2 = re.search(r'\n',lines[current_start_index + index_init1:-1]).span()[1]
        index_init =  current_start_index + index_init1 + index_init2

        index_finit1 = re.search(r'free_viewing_experiment_end',lines[index_init:-1]).span()[0]
        index_finit = index_init + index_finit1

        indices.append([index_init, index_finit-1])
        current_start_index = index_finit

        # if(i + 1) % 5 == 0:
        #     j = int(j/2)

        new_vec = lines[index_init: index_finit-1].split("\n")
        new_new_vec = []
        #0 = blink, 1 fixation, 2 saccade
        stateL = 0
        stateR = 0
        for k in range(1,len(new_vec)):
            la = new_vec[k].split("\t")
            if len(la) == 1:
                
                if 'SSACC L' in la[0]:
                    stateL = 2
                if 'SFIX L' in la[0]:
                    stateL = 1
                if 'SBLINK L' in la[0]:
                    stateL = 0
                if 'SSACC R' in la[0]:
                    stateR = 2
                if 'SFIX R' in la[0]:
                    stateR = 1
                if 'SBLINK R' in la[0]:
                    stateR = 0
            if len(la) == 9:
                if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                    continue
                else:
                    la[-1] = stateL
                    la.append(stateR)
                    new_new_vec.append(la)
            if len(la) == 6:
                if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                    continue
                else:
                    la[-1] = stateL
                    la.append(stateR)
                    new_new_vec.append(la)

        ls = [[np.nan if x == '   .' else x for x in y] for y in new_new_vec]
        if len(ls)>0:
            ls = np.swapaxes(ls,0,1)
            current_trial = np.asarray(ls, dtype=float)
            # if participant =='P205':
            # print(len(current_trial))
            if len(current_trial) == 10:
                my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[4][:],current_trial[5][:],current_trial[-1][:]]),0,1)), columns=['Time','RX','RY','state'])
            else:
                # print(current_trial[7][:])
                my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[1][:],current_trial[2][:],current_trial[5][:]]),0,1)), columns=['Time','RX','RY','state'])
            my_df['participants'] = participant
            interval_num = np.minimum((np.arange(len(my_df)) // 3000) + 1, 40)
            my_df['interval_num'] = interval_num
            # my_df.to_csv("./free_viewing/eyelink_data/" + participant + "_" + str(interval_num) + ".csv", index=False)
            # my_df['FREQ']=j

            task_data_df.append(my_df)
            df_final = pd.concat([df_final,my_df], ignore_index=True)
        else:
            not_list.append(i)


df_final.to_csv("../../processed_data/processed_data_asc_task_free_viewing.csv", index=False)
    

P006
P008
P020
P036
P063
P065
P070
P072
P074
P076
P086
P094
P103
P106
P109
P112
P150
P157
P160
P170
P171
P172
P173
P176
P177
P182
P183
P186
P188
P190
P194
P201
P202
P203
P204
P205
P207
P208
P210


In [6]:
df_final

,Time,RX,RY,state,participants,interval_num
0,50662247.0,NaN,NaN,0.0,P006,1
1,50662248.0,NaN,NaN,0.0,P006,1
2,50662249.0,NaN,NaN,0.0,P006,1
3,50662250.0,NaN,NaN,0.0,P006,1
4,50662251.0,NaN,NaN,0.0,P006,1
...,...,...,...,...,...,...
4682072,428301439.0,750.4,592.0,2.0,P210,40
4682073,428301440.0,756.0,592.0,2.0,P210,40
4682074,428301441.0,764.2,592.2,2.0,P210,40
4682075,428301442.0,773.2,592.4,2.0,P210,40


In [9]:
grouped = df_final.groupby(['participants', 'interval_num'])

In [11]:
groups_dict = {}
for (participant, interval), group in grouped:
    key = f"{participant}_{interval}"
    groups_dict[key] = group
    group.to_csv(f"./free_viewing/eyelink_data/{key}.csv", index=False)

In [1]:
import re
import os
import numpy as np
import pandas as pd
from pathlib import Path

def process_faces_interval_block(block_text, participant, interval_num):
    """
    Process a block of text corresponding to one free viewing (faces) interval.
    This function mimics the state‐processing from your ASC saccade code:
      - Splits the block into lines.
      - For each line, if it has one field (no tab), it updates state markers for left and right eyes.
      - For lines with 9 fields (data lines), it replaces the last field with the current left state,
        appends the right state, and collects these.
    Returns a DataFrame with columns: Time, RX, RY, state, plus added participant and interval_num.
    """
    lines = block_text.split("\n")
    processed_lines = []
    
    # Initialize state markers: (0=blink, 1=fixation, 2=saccade)
    stateL = 0
    stateR = 0

    # Process lines in the block (skip first line if header-like)
    for line in lines:
        # Split by tab.
        fields = line.split("\t")
        # If the line is a single field, update states:
        if len(fields) == 1:
            text = fields[0].strip().lower()
            if 'ssacc l' in text:
                stateL = 2
            elif 'sfix l' in text:
                stateL = 1
            elif 'sblink l' in text:
                stateL = 0
            if 'ssacc r' in text:
                stateR = 2
            elif 'sfix r' in text:
                stateR = 1
            elif 'sblink r' in text:
                stateR = 0
        # If the line has 9 fields, assume it is a data line.
        elif len(fields) == 9:
            # Optionally, ignore lines with markers like 'esacc', 'efix', or 'eblink'
            if any(marker in fields[0].lower() for marker in ['esacc l', 'efix l', 'eblink l',
                                                              'esacc r', 'efix r', 'eblink r']):
                continue
            # Otherwise, update the fields: replace the last field with stateL and append stateR.
            fields[-1] = str(stateL)
            fields.append(str(stateR))
            processed_lines.append(fields)
    
    # If no data lines were processed, return an empty DataFrame.
    if not processed_lines:
        return pd.DataFrame()
    
    # Convert processed_lines to a numpy array and swap axes so that each row becomes one record.
    arr = np.array(processed_lines)
    # In your saccade code you assumed:
    # arr[0][:] -> Time, arr[4][:] -> RX, arr[5][:] -> RY, and last row -> state.
    # Adjust these indices if necessary; here we assume the same.
    try:
        # Extract columns and swap axes: each record becomes [Time, RX, RY, state]
        data = np.swapaxes(np.array([arr[0, :].astype(float), arr[4, :].astype(float), 
                                      arr[5, :].astype(float), arr[-1, :].astype(float)]), 0, 1)
    except Exception as e:
        print("Error processing block to array:", e)
        return pd.DataFrame()
    
    df_interval = pd.DataFrame(data, columns=['Time', 'RX', 'RY', 'state'])
    df_interval['participants'] = participant
    df_interval['interval_num'] = interval_num
    return df_interval

def process_faces_asc_file(asc_file):
    """
    Process a faces ASC file for one participant.
    
    The ASC file contains both calibrated data and embedded messages.
    This function:
      1. Reads the entire file.
      2. Extracts the global free viewing experiment boundaries using markers:
            free_viewing_experiment_start and free_viewing_experiment_end.
      3. Divides the block between these markers into 40 equal parts.
      4. For each subinterval, processes the block to extract data and state (via process_faces_interval_block).
      5. Returns a concatenated DataFrame with all intervals.
    """
    with open(asc_file, 'r') as f:
        full_text = f.read()
    
    # Extract global free viewing experiment boundaries.
    start_match = re.search(r'free_viewing_experiment_start', full_text, re.IGNORECASE)
    end_match = re.search(r'free_viewing_experiment_end', full_text, re.IGNORECASE)
    if not start_match or not end_match:
        print(f"Free viewing boundaries not found in {asc_file}")
        return pd.DataFrame()
    
    # Get the indices for the experiment block.
    exp_start_idx = start_match.end()
    exp_end_idx = end_match.start()
    exp_text = full_text[exp_start_idx:exp_end_idx]
    
    # Divide the experiment block into 40 intervals.
    total_length = len(exp_text)
    interval_count = 40
    interval_length = total_length // interval_count  # integer division
    
    merged_intervals = []
    # Extract participant ID.
    participant_match = re.search(r"(P\d+)", os.path.basename(asc_file))
    participant = participant_match.group(1) if participant_match else ""
    
    for i in range(interval_count):
        # Compute start and end indices in the experiment text.
        start_idx = i * interval_length
        # For the last interval, take until the end.
        if i == interval_count - 1:
            end_idx = total_length
        else:
            end_idx = (i + 1) * interval_length
        interval_block = exp_text[start_idx:end_idx]
        df_interval = process_faces_interval_block(interval_block, participant, i + 1)
        if not df_interval.empty:
            merged_intervals.append(df_interval)
    
    if merged_intervals:
        return pd.concat(merged_intervals, ignore_index=True)
    else:
        return pd.DataFrame()

def main():
    # Collect ASC files for faces task.
    # Adjust the folder path as needed.
    asc_files = list(Path('./sample_data').rglob('*.asc'))
    df_all = pd.DataFrame()
    
    for asc_file in asc_files:
        df_participant = process_faces_asc_file(str(asc_file))
        if not df_participant.empty:
            df_all = pd.concat([df_all, df_participant], ignore_index=True)
    
    if not df_all.empty:
        df_all = df_all.sort_values(['participants', 'Time']).reset_index(drop=True)
        df_all.to_csv("./processed_data/processed_data_asc_task_free_viewing.csv", index=False)
        print("Merged Free Viewing (Faces) ASC Data:")
        print(df_all.head(20))
    else:
        print("No data merged.")

if __name__ == "__main__":
    main()


Error processing block to array: could not convert string to float: '   .'
Merged Free Viewing (Faces) ASC Data:
    Time   RX   RY  state participants  interval_num
0    0.0  0.0  0.0    0.0         P184             1
1    0.0  0.0  0.0    1.0         P184             1
2    0.0  0.0  0.0    1.0         P184             1
3    0.0  0.0  0.0    0.0         P184             2
4    0.0  0.0  0.0    2.0         P184             2
5    0.0  0.0  0.0    2.0         P184             2
6    0.0  0.0  0.0    0.0         P184             3
7    0.0  0.0  0.0    1.0         P184             3
8    0.0  0.0  0.0    1.0         P184             3
9    0.0  0.0  0.0    0.0         P184             4
10   0.0  0.0  2.0    1.0         P184             4
11   0.0  2.0  2.0    1.0         P184             4
12   0.0  0.0  0.0    0.0         P184             5
13   0.0  0.0  0.0    1.0         P184             5
14   0.0  0.0  0.0    1.0         P184             5
15   0.0  0.0  0.0    0.0         P184 